# 07 - FX Option Pricing (Garman-Kohlhagen)

Price FX options, verify parity, and optionally invert implied volatility.

In [3]:
%cd /content
!rm -rf interactive_portfolio_optimization
!git clone https://github.com/basarr/interactive_portfolio_optimization.git
!ls /content/interactive_portfolio_optimization

/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 147 (delta 73), reused 51 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 100.35 KiB | 2.23 MiB/s, done.
Resolving deltas: 100% (73/73), done.
data  notebooks  pyproject.toml  README.md  results  sources  src  tests


In [5]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if not (ROOT / "src").exists():
    raise FileNotFoundError(f"Bad ROOT: {ROOT}")

for p in [ROOT/"results", ROOT/"results"/"tables", ROOT/"results"/"figures", ROOT/"results"/"logs", ROOT/"results"/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT OK:", ROOT)

ROOT OK: /content/interactive_portfolio_optimization


In [6]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.fx_options import gk_call_price, gk_put_price, gk_put_call_parity_residual, gk_implied_vol

cfg = config_dict(fast_mode=True)


In [10]:
from pathlib import Path

# Re-assert the correct ROOT path, as it was overwritten by a previous cell.
# This assumes the interactive_portfolio_optimization directory is in /content.
ROOT = Path('/content/interactive_portfolio_optimization')

# Ensure the results/tables directory exists under this ROOT
(ROOT / 'results' / 'tables').mkdir(parents=True, exist_ok=True)

strikes = [1.00, 1.05, 1.10, 1.15, 1.20]
rows = []
for K in strikes:
    call = gk_call_price(cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['SIGMA_FX'], cfg['T_FX'])
    put = gk_put_price(cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['SIGMA_FX'], cfg['T_FX'])
    residual = gk_put_call_parity_residual(cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['SIGMA_FX'], cfg['T_FX'])
    iv = gk_implied_vol(call, cfg['S0_FX'], K, cfg['RD'], cfg['RF'], cfg['T_FX'], 'call')
    rows.append({'strike': K, 'call': call, 'put': put, 'parity_residual': residual, 'implied_vol_call': iv})

fx_df = pd.DataFrame(rows)
fx_df.to_csv(ROOT / 'results' / 'tables' / 'fx_parity_checks.csv', index=False)
fx_df

,strike,call,put,parity_residual,implied_vol_call
0,1.00,0.129663,0.011054,0.000000e+00,0.12
1,1.05,0.093015,0.022928,0.000000e+00,0.12
2,1.10,0.063086,0.041521,1.110223e-16,0.12
3,1.15,0.040361,0.067319,0.000000e+00,0.12
4,1.20,0.024351,0.099831,0.000000e+00,0.12
